In [ ]:
import time
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import utils

In [2]:
graph_short_names = ["Karate", "Student", "Jazz", "FB", "FB1", "NetSci", "ER1", "ER2"]

In [3]:
# load data
with open("data/graph_adj_csr.pkl", "rb") as f:
    data = pickle.load(f)

# adjacency matrices (scipy.sparse._csr.csr_matrix)
karate = data["Karate"]
student = data["Student"]
jazz = data["Jazz"]
facebook = data["FB"]
FB1 = data["FB1"]
netsci = data["NetSci"]

df = pd.read_csv("out/network_stats.csv", index_col=0)
df.index = graph_short_names[:-2]
df

,Nodes,Edges,density,<k>,k_std,k_max,diameter
Karate,34.0,78.0,0.139037,4.588235,0.655186,17.0,5.0
Student,141.0,297.0,0.030091,4.212766,0.139991,10.0,17.0
Jazz,198.0,2742.0,0.140594,27.696970,1.237300,100.0,6.0
FB,329.0,1954.0,0.036215,11.878419,0.603350,63.0,9.0
FB1,320.0,2369.0,0.046415,14.806250,0.797175,113.0,7.0
NetSci,379.0,914.0,0.012760,4.823219,0.201725,34.0,17.0


In [4]:
N = 100
p = 0.07 # to ensure connectivity and spectral condition

G_er1 = nx.fast_gnp_random_graph(N, p, seed=123)
G_er2 = nx.fast_gnp_random_graph(N, 5*p, seed=123)

df.loc["ER1"] = utils.network_stats(G_er1)
df.loc["ER2"] = utils.network_stats(G_er2)
df

,Nodes,Edges,density,<k>,k_std,k_max,diameter
Karate,34.0,78.0,0.139037,4.588235,0.655186,17.0,5.0
Student,141.0,297.0,0.030091,4.212766,0.139991,10.0,17.0
Jazz,198.0,2742.0,0.140594,27.696970,1.237300,100.0,6.0
FB,329.0,1954.0,0.036215,11.878419,0.603350,63.0,9.0
FB1,320.0,2369.0,0.046415,14.806250,0.797175,113.0,7.0
NetSci,379.0,914.0,0.012760,4.823219,0.201725,34.0,17.0
ER1,100.0,367.0,0.074141,7.340000,NaN,15.0,5.0
ER2,100.0,1755.0,0.354545,35.100000,NaN,46.0,2.0


In [5]:
# Get spectrum of each graph
matrices = {
    "Karate": karate.toarray(),
    "Student": student.toarray(),
    "Jazz": jazz.toarray(),
    "FB": facebook.toarray(),
    "FB1": FB1.toarray(),
    "NetSci": netsci.toarray(),
    "ER1": nx.to_numpy_array(G_er1),
    "ER2": nx.to_numpy_array(G_er2),
}

eigenvalues = {}

start = time.time()
for name, mat in matrices.items():
    try:
        eigenvalues[name] = np.linalg.eigvalsh(mat)
    except Exception as e:
        print(f"Error with {name}: {e}")
        eigenvalues[name] = None
end = time.time()
print(f"Time taken to compute eigenvalues: {end - start:.2f} seconds")

Time taken to compute eigenvalues: 0.04 seconds


In [6]:
# Get tolerance for zero eigenvalues
start = time.time()
tolerances = {name: utils.zero_tolerance(mat) for name, mat in matrices.items()}
end = time.time()
print(f"Time taken to compute tolerances: {end - start:.2f} seconds")

Time taken to compute tolerances: 0.10 seconds


In [7]:
df_spectrum = df.copy()
start = time.time()
for name, eigs in eigenvalues.items():
    if eigs is not None:
        df_spectrum.loc[name, "min_eigenvalue"] = np.min(eigs)
        df_spectrum.loc[name, "spectral_radius"] = np.max(np.abs(eigs))
        df_spectrum.loc[name, "has_near_zero"] = np.any(np.isclose(eigs, 0.0, atol=tolerances[name]))
        df_spectrum.loc[name, "rank"] = np.linalg.matrix_rank(matrices[name], hermitian=True)        # Return matrix rank of array using SVD method
        df_spectrum.loc[name, "is_full_rank"] = utils.is_full_rank(matrices[name], hermitian=True)        # Return matrix rank of array using SVD method
    else:
        df_spectrum.loc[name] = [None, None, None]
end = time.time()
print(f"Time taken to compute spectrum DataFrame: {end - start:.2f} seconds")
df_spectrum

Time taken to compute spectrum DataFrame: 0.10 seconds


,Nodes,Edges,density,<k>,k_std,k_max,diameter,min_eigenvalue,spectral_radius,has_near_zero,rank,is_full_rank
Karate,34.0,78.0,0.139037,4.588235,0.655186,17.0,5.0,-4.487229,6.725698,True,24.0,False
Student,141.0,297.0,0.030091,4.212766,0.139991,10.0,17.0,-4.178497,6.311929,False,141.0,True
Jazz,198.0,2742.0,0.140594,27.696970,1.237300,100.0,6.0,-8.702641,40.027376,False,198.0,True
FB,329.0,1954.0,0.036215,11.878419,0.603350,63.0,9.0,-7.490927,24.446709,True,324.0,False
FB1,320.0,2369.0,0.046415,14.806250,0.797175,113.0,7.0,-9.605722,29.729055,False,320.0,True
NetSci,379.0,914.0,0.012760,4.823219,0.201725,34.0,17.0,-5.469570,10.375459,True,358.0,False
ER1,100.0,367.0,0.074141,7.340000,NaN,15.0,5.0,-5.313278,8.525764,False,100.0,True
ER2,100.0,1755.0,0.354545,35.100000,NaN,46.0,2.0,-9.039754,35.770712,False,100.0,True


In [8]:
# Compute values of welfare ratio for each graph at attenuation=0.01
start = time.time()
welfare_values = utils.candidate_welfare_values(attenuation=0.01, eigenvalues=eigenvalues)
end = time.time()
print(f"Time taken to compute welfare values: {end - start:.2f} seconds")

Time taken to compute welfare values: 0.00 seconds


In [9]:
# Compute welfare bounnds (LU & UB)
start = time.time()
for k, v in welfare_values.items():
    if v is not None:
        print(f"{k}: min={np.min(v):.5f}, max={np.max(v): .20f}", "Exact one?: ", np.max(v) == 1.0)
        df_spectrum.loc[k, "LB"] = np.min(v)
        df_spectrum.loc[k, "UB"] = np.max(v)
end = time.time()
print("===========================================")
print(f"Time taken to compute welfare bounds: {end - start:.2f} seconds")
df_spectrum

Karate: min=0.99480, max= 1.00000000000000000000 Exact one?:  True
Student: min=0.99546, max= 0.99999999005174233790 Exact one?:  False
Jazz: min=0.55454, max= 0.99999998230467534999 Exact one?:  False
FB: min=0.89530, max= 1.00000000000000000000 Exact one?:  True
FB1: min=0.82102, max= 0.99999999662879013318 Exact one?:  False
NetSci: min=0.98660, max= 1.00000000000000000000 Exact one?:  True
ER1: min=0.99131, max= 0.99999999820805685058 Exact one?:  False
ER2: min=0.68984, max= 0.99999966729926603737 Exact one?:  False
Time taken to compute welfare bounds: 0.00 seconds


,Nodes,Edges,density,<k>,k_std,k_max,diameter,min_eigenvalue,spectral_radius,has_near_zero,rank,is_full_rank,LB,UB
Karate,34.0,78.0,0.139037,4.588235,0.655186,17.0,5.0,-4.487229,6.725698,True,24.0,False,0.994801,1.0
Student,141.0,297.0,0.030091,4.212766,0.139991,10.0,17.0,-4.178497,6.311929,False,141.0,True,0.995461,1.0
Jazz,198.0,2742.0,0.140594,27.696970,1.237300,100.0,6.0,-8.702641,40.027376,False,198.0,True,0.554541,1.0
FB,329.0,1954.0,0.036215,11.878419,0.603350,63.0,9.0,-7.490927,24.446709,True,324.0,False,0.895303,1.0
FB1,320.0,2369.0,0.046415,14.806250,0.797175,113.0,7.0,-9.605722,29.729055,False,320.0,True,0.821017,1.0
NetSci,379.0,914.0,0.012760,4.823219,0.201725,34.0,17.0,-5.469570,10.375459,True,358.0,False,0.986598,1.0
ER1,100.0,367.0,0.074141,7.340000,NaN,15.0,5.0,-5.313278,8.525764,False,100.0,True,0.991313,1.0
ER2,100.0,1755.0,0.354545,35.100000,NaN,46.0,2.0,-9.039754,35.770712,False,100.0,True,0.689838,1.0


In [10]:
# for paper
df_paper = df_spectrum.drop(columns=["Edges", "k_std", "diameter", "min_eigenvalue", "has_near_zero", "is_full_rank"])
df_paper.to_csv("out/welfare_bounds.csv")
df_paper

,Nodes,density,<k>,k_max,spectral_radius,rank,LB,UB
Karate,34.0,0.139037,4.588235,17.0,6.725698,24.0,0.994801,1.0
Student,141.0,0.030091,4.212766,10.0,6.311929,141.0,0.995461,1.0
Jazz,198.0,0.140594,27.696970,100.0,40.027376,198.0,0.554541,1.0
FB,329.0,0.036215,11.878419,63.0,24.446709,324.0,0.895303,1.0
FB1,320.0,0.046415,14.806250,113.0,29.729055,320.0,0.821017,1.0
NetSci,379.0,0.012760,4.823219,34.0,10.375459,358.0,0.986598,1.0
ER1,100.0,0.074141,7.340000,15.0,8.525764,100.0,0.991313,1.0
ER2,100.0,0.354545,35.100000,46.0,35.770712,100.0,0.689838,1.0
